<a href="https://colab.research.google.com/github/SaddamHasanov/Sentiment-Analysis-Project/blob/main/Sentiment_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performing sentiment analysis on product comments taken from the "e-Ticaret" site

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# if emoji module not defined use this command to install the 1.4.1 version of emoji module
!pip install emoji==1.4.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.1/185.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-1.4.1-py3-none-any.whl size=186377 sha256=c79421ed9837285b5f0e6e9d5cc9279b4439c49c9cc9f5c564bdaf2ee71f5ff4
  Stored in directory: /root/.cache/pip/wheels/35/27/d6/a425b9b845119a8e2f1fb85f405bc4ba2a836bd2805c3c8403
Successfully built emoji


# Importing Libraries

In [ ]:
import pandas as pd
import string
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import emoji
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report
from sklearn.svm import LinearSVC
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Getting Data And Clean It

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis_Project_For_Ufaz/Raw_Data/sentiment_data.csv')
data

,Unnamed: 0,Rating,Review
0,0,1.0,3 yıldır tık demedi. :)
1,1,1.0,3 yıldır kullanıyorum müthiş
2,2,1.0,Ürün bugün elime geçti çok fazla inceleme fırs...
3,3,1.0,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,4,1.0,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...,...
281937,4528,0.0,turkcell'den 2000 yılında new york borsasında ...
281938,4529,0.0,Yeni telefon sapığım Turkcell .Açmadığım halde...
281939,4530,0.0,106 lira fatura mı olur turkcell sadece soruyorum
281940,4531,0.0,@Buragl111 turkcell kazık takıl sen bimcell le...


In [ ]:
data = data.sample(frac = 0.2, random_state = 9999)
data

,Unnamed: 0,Rating,Review
56245,56245,1.0,3 yıl önce başka bir yerden almıştım.mükemmel ...
280196,2787,0.0,Sevdiklerimle dakika yapmadan rahat rahat konu...
218737,218737,0.0,Sık sık kesinti olduğu için başka bir dörtlü l...
113742,113742,1.0,4 gün önce sipariş verdim ve bügün bana saat 1...
129742,129742,1.0,"Mükemmel ürün, mükemmel fiyat, mükemmel kargo."
...,...,...,...
193520,193520,1.0,"ürünü daha öncede almıştım, kedilerimiz severe..."
177101,177101,1.0,Kargo hızlı fiyat güzel kalite zaten ortada tş...
142923,142923,1.0,hepsiburadaya hızlı gönderimden dolayı teşekkü...
128109,128109,1.0,Anneme aldım çok teşekkürler. Memnun kaldı.


Drop Useless Column

In [ ]:
data = data.drop('Unnamed: 0', axis = 1)
data

,Rating,Review
56245,1.0,3 yıl önce başka bir yerden almıştım.mükemmel ...
280196,0.0,Sevdiklerimle dakika yapmadan rahat rahat konu...
218737,0.0,Sık sık kesinti olduğu için başka bir dörtlü l...
113742,1.0,4 gün önce sipariş verdim ve bügün bana saat 1...
129742,1.0,"Mükemmel ürün, mükemmel fiyat, mükemmel kargo."
...,...,...
193520,1.0,"ürünü daha öncede almıştım, kedilerimiz severe..."
177101,1.0,Kargo hızlı fiyat güzel kalite zaten ortada tş...
142923,1.0,hepsiburadaya hızlı gönderimden dolayı teşekkü...
128109,1.0,Anneme aldım çok teşekkürler. Memnun kaldı.


Find All NULL Values And Drop Them

In [ ]:
data.isnull().sum()

Rating    91
Review     1
dtype: int64

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

Rating    0
Review    0
dtype: int64

Let's look at the Rating column

In [ ]:
data['Rating'].value_counts()

Rating
1.0    50513
0.0     5783
Name: count, dtype: int64

As you can see 253058 rows of this column are 1.0 and others are 0.0. 1.0 means **positive** comments and 0.0 means **negative** comments. Let's change the type of this column from float to **integer**

In [ ]:
data['Rating'] = data['Rating'].astype(int)

<ipython-input-16-5925a877f1cd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Rating'] = data['Rating'].astype(int)


In [ ]:
data.dtypes

Rating     int64
Review    object
dtype: object

# Removing Emojis With Example

In [ ]:
# Example
text = 'Hi, My name is Nicat. \U0001f602' # text with emoji and punctuations
print(text)
text = emoji.get_emoji_regexp().sub(u'', text)
print(text)

# Apply it to Review column
data.loc[:, 'Review'] = data.loc[:, 'Review'].apply(lambda x: emoji.get_emoji_regexp().sub(u'', x))

Hi, My name is Nicat. 😂
Hi, My name is Nicat. 


# Removing Punctuations
mapping every character from string.punctuation to None and then using string.translate method to remove all punctuation marks from each word of the text

In [ ]:
# punctuation attribute from string library
print(string.punctuation)
print('\n')
# mapping all punctuation marks to None
print(str.maketrans('', '', string.punctuation))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [ ]:
# removing all punctuation marks from each word of the text
data.loc[:, 'Review'] = data.loc[:, 'Review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# Changing Uppercase Letters To Lowercase

In [ ]:
data.loc[:, 'Review'] = data.loc[:, 'Review'].apply(lambda x: x.lower())

# Info About Stop Words And Its Application

Stop Words removal:

When we use the features from a text to model, we will encounter a lot of noise. These are the stop words like **the**, **he**, **her**, etc… in english which don’t help us and, just be removed before processing for cleaner processing inside the model. Also there are many stop words in turkish language. With NLTK we can see all the stop words available in the **Turkish** language. (like : ['**biz**', '**o**', '**bu**'...])

In [ ]:
print(stopwords.words("turkish"))

['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani']


In [ ]:
stop_words = set(stopwords.words('turkish'))
data['Review'] = data["Review"].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

<ipython-input-22-7eb4cba4ecc3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Review'] = data["Review"].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))


# Removing Any Leading And trailing whitespaces

In [ ]:
data.loc[:, 'Review'] = data.loc[:, 'Review'].apply(lambda x: str.strip(x))

# Split Data Into Train And Validation Data Sets

Firs Separate Input and Output Columns (Review and Rating)

In [ ]:
# Input
x = data['Review']
# Output
y = data['Rating']

In [ ]:
x

56245     3 yıl önce başka bir yerden almıştımmükemmel b...
280196    sevdiklerimle dakika yapmadan rahat rahat konu...
218737    sık sık kesinti olduğu başka bir dörtlü lnb ku...
113742    4 gün önce sipariş verdim bügün bana saat 1 el...
129742          mükemmel ürün mükemmel fiyat mükemmel kargo
                                ...                        
193520    ürünü öncede almıştım kedilerimiz severek yiyo...
177101    kargo hızlı fiyat güzel kalite zaten ortada tş...
142923    hepsiburadaya hızlı gönderimden dolayı teşekkü...
128109                anneme aldım teşekkürler memnun kaldı
193675    mahallemde sokaktaki diğer kediler aldım sıkın...
Name: Review, Length: 56296, dtype: object

In [ ]:
y

56245     1
280196    0
218737    0
113742    1
129742    1
         ..
193520    1
177101    1
142923    1
128109    1
193675    1
Name: Rating, Length: 56296, dtype: int64

# Creating The Count Vectorizer

In [ ]:
# Transform each text into a vector of word counts
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(x)

NameError: name 'CountVectorizer' is not defined

# Split Data Into Train And Validation Data Sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.1, shuffle = True)

# Training The Model

I use Linear Support Vector Classifier to train this model

In [ ]:
model = LinearSVC()
model.fit(x_train, y_train)

LinearSVC()

Then I use test or validation data to test my model

In [ ]:
y_pred = model.predict(x_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1])

# Model Evaluation

In [ ]:
accuracy_score(y_test, y_pred)

0.9287744227353464

Accuracy of our model is about 93 %. This is very high and good accuracy, but still not enough. Let's look at confusion_matrix and f1-score to check wheather we have some overfitting problems or not.

In [ ]:
confusion_matrix(y_test, y_pred)

array([[ 306,  258],
       [ 143, 4923]])

As you can see, in **confusion matrix** **4923** values are True Positive and **306** values are True negative. That means our model classified 5229 (4923 + 306) reviews right and 401 (143 + 258) reviews wrong.

In [ ]:
f1_score(y_test, y_pred)

0.9608665951010051

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.68      0.54      0.60       564
           1       0.95      0.97      0.96      5066

    accuracy                           0.93      5630
   macro avg       0.82      0.76      0.78      5630
weighted avg       0.92      0.93      0.93      5630



So, **precision**, **recall** and **f1-score** in both classes are close to each other. As a result of our model evaluation we can say that, there is no overfitting problem, but in our dataset we have very few negative comments (0) compared to the positive comments (1), so our dataset is not balanced. For these reasons sometimes our model can't classify negative comments right, but in general model works well.

# Sample Testing

In [ ]:
sample = pd.DataFrame({
  'Review':['Harika ürün, Gerçekden çox beyendim muhteşem olmuş',
            'turkcell beni yine ayar etti.',
            'En kötü ürünlerden bi tanesi daha']
})

In [ ]:
z = vectorizer.transform(sample["Review"])

answer_list = model.predict(z)

for answer in answer_list:
  if answer == 0:
    print('Negative')
  else:
    print('Positive')

Positive
Negative
Negative
